# About This Notebook

<p style='font-family: Segoe UI; font-size: 1.5em; font-weight: 400; font-size: 15px'> This implementation is based on a vanilla <b>swin_large_patch4_window12_384</b> and <b>efficientnet_b4</b> in Pytorch for the Pawpularity Competition.<br>
This model uses <b>both images and dense features</b> for score prediction.<br>
<b>This scores around 18.1 LB.</b></p>

<p style='color: #fc0362; font-family: Segoe UI; font-size: 1.5em; font-weight: 400; font-size: 20px'>Here I have attempted to use a Vision transformer model and a CNN model in conjunction. The embeddings from both the Swin and EfficientNet model are concatenated along with the dense features before passing through a 2 layer fully connected network.</p>

<p style='font-family: Segoe UI; font-size: 1.5em; font-weight: 400; font-size: 18px'>
Training Params: -
<ol style='font-family: Segoe UI; font-size: 1.5em; font-weight: 400; font-size: 15px'>
<li> <b>Dataset</b>: - 3-channel RGB Images (384x384) with separate dense features </li>
<li> <b>Augmentations</b>: - Resize, Normalize, HorizontalFlip, VerticalFlip, RandomBrightness, RandomResizedCrop, HueSaturationValue, RandomBrightnessContrast </li>
<li> <b>Optimizer</b>: - AdamW </li>
<li> <b>Scheduler</b>: - CosineAnnealingLR </li>
<li> <b>Model</b>: - swin_large_patch4_window12_384, efficientnet_b4 </li>
<li> <b>Initial Weights</b>: - Imagenet </li>
<li> <b>Max Epochs</b>: - 8 (~29 min per epoch on P100 PCIE GPU) </li>
<li> <b>Saved Weights</b>: - 10-fold ensemble. Weights having highest OOF score on RMSE metric were saved. </li>
</ol>
</p>
<p style='font-family: Segoe UI; font-size: 1.5em; font-weight: 400; font-size: 15px'>
This notebook only contains the inference for the model as described above.<br><br>
If you are looking for a starter training notebook please follow the link below: -<br></p>
<ul style='font-family: Segoe UI; font-size: 1.5em; font-weight: 400; font-size: 15px'>
<li>Baseline Model Notebook:- <a href=https://www.kaggle.com/manabendrarout/pawpularity-score-starter-image-dense-train>Training Notebook</a></li>
</ul>
<p style='font-family: Segoe UI; font-size: 1.5em; font-weight: 400; font-size: 15px'>
<b>NB:-</b> This training notebook uses a different NN architecture. Not the exact architecture used for this notebook. But apart from the architecture, everything else (training parameters, optimizers, schedulers, etc) is same. I had to use a different architecture for demonstration because Kaggle has a timeout limit which is not possible to adhere with the transformer model.
</p>

![SETI](https://www.petfinder.my/images/cuteness_meter.jpg)  

**If you find this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels. 😊**

# Get GPU Info

In [ ]:
!nvidia-smi

# Import

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import os
import glob
import random
import cv2
pd.set_option('display.max_columns', None)

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Machine Learning
from xgboost import XGBRegressor

# Random Seed Initialize
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
csv_dir = '../input/petfinder-pawpularity-score'
test_dir = '../input/petfinder-pawpularity-score/test'
models_dir = '../input/pawpularity-contest-models/Swin_large_EfficientnetB4'

test_file_path = os.path.join(csv_dir, 'test.csv')
sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv')
print(f'Test file: {test_file_path}')
print(f'Models path: {models_dir}')

In [ ]:
test_df = pd.read_csv(test_file_path)
sample_df = pd.read_csv(sample_sub_file_path)

In [ ]:
def return_filpath(name, folder):
    path = os.path.join(folder, f'{name}.jpg')
    return path

In [ ]:
test_df['image_path'] = test_df['Id'].apply(lambda x: return_filpath(x, folder=test_dir))

In [ ]:
test_df.head()

In [ ]:
target = ['Pawpularity']
not_features = ['Id', 'kfold', 'image_path', 'Pawpularity']
cols = list(test_df.columns)
features = [feat for feat in cols if feat not in not_features]
print(features)

# CFG

In [ ]:
params = {
    'model_1': 'swin_large_patch4_window12_384',
    'model_2': 'efficientnet_b4',
    'dense_features': features,
    'pretrained': False,
    'inp_channels': 3,
    'im_size': 384,
    'device': device,
    'batch_size': 8,
    'num_workers' : 2,
    'out_features': 1,
    'debug': False
}

In [ ]:
if params['debug']:
    test_df = test_df.sample(frac=0.1)

# Augmentations

In [ ]:
def get_test_transforms(DIM = params['im_size']):
    return albumentations.Compose(
        [
          albumentations.Resize(DIM,DIM),
          albumentations.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225],
          ),
          ToTensorV2(p=1.0)
        ]
    )

# Dataset

In [ ]:
class CuteDataset(Dataset):
    def __init__(self, images_filepaths, dense_features, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.dense_features = dense_features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        dense = self.dense_features[idx, :]
        label = torch.tensor(self.targets[idx]).float()
        return image, dense, label

# CNN Model

In [ ]:
class PetNet(nn.Module):
    def __init__(self, model_1_name=params['model_1'], model_2_name=params['model_2'],
                 out_features=params['out_features'], inp_channels=params['inp_channels'],
                 pretrained=params['pretrained'], num_dense=len(params['dense_features'])):
        super().__init__()
        
        # Transformer
        self.model_1 = timm.create_model(model_1_name, pretrained=pretrained,
                                         in_chans=inp_channels)
        n_features_1 = self.model_1.head.in_features
        self.model_1.head = nn.Linear(n_features_1, 128)
        
        # Conventional CNN
        self.model_2 = timm.create_model(model_2_name, pretrained=pretrained,
                                         in_chans=inp_channels)
        out_channels = self.model_2.conv_stem.out_channels
        kernel_size = self.model_2.conv_stem.kernel_size
        stride = self.model_2.conv_stem.stride
        padding = self.model_2.conv_stem.padding
        bias = self.model_2.conv_stem.bias
        self.model_2.conv_stem = nn.Conv2d(inp_channels, out_channels,
                                           kernel_size=kernel_size,
                                           stride=stride, padding=padding,
                                           bias=bias)
        n_features_2 = self.model_2.classifier.in_features
        self.model_2.classifier = nn.Linear(n_features_2, 128)
        
        self.fc = nn.Sequential(
            nn.Linear(128 + 128 + num_dense, 64),
            nn.ReLU(),
            nn.Linear(64, out_features)
        )
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, image, dense):
        transformer_embeddings = self.model_1(image)
        conv_embeddings = self.model_2(image)
        features = torch.cat([transformer_embeddings, conv_embeddings, dense],
                             dim=1)
        x = self.dropout(features)
        output = self.fc(x)
        return output

# Prediction

In [ ]:
predictions_nn = None
for model_name in glob.glob(models_dir + '/*.pth'):
    model = PetNet()
    model.load_state_dict(torch.load(model_name))
    model = model.to(params['device'])
    model.eval()

    test_dataset = CuteDataset(
        images_filepaths = test_df['image_path'].values,
        dense_features = test_df[params['dense_features']].values,
        targets = sample_df['Pawpularity'].values,
        transform = get_test_transforms()
    )
    test_loader = DataLoader(
        test_dataset, batch_size=params['batch_size'],
        shuffle=False, num_workers=params['num_workers'],
        pin_memory=True
    )

    temp_preds = None
    with torch.no_grad():
        for (images, dense, target) in tqdm(test_loader, desc=f'Predicting. '):
            images = images.to(params['device'], non_blocking=True)
            dense = dense.to(params['device'], non_blocking=True)
            predictions = torch.sigmoid(model(images, dense)).to('cpu').numpy()*100
            
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))
                
    test_df[model_name.split('/')[-1].split('_')[-3]] = temp_preds

    if predictions_nn is None:
        predictions_nn = temp_preds
    else:
        predictions_nn += temp_preds
        
predictions_nn /= (len(glob.glob(models_dir + '/*.pth')))

# Submission

In [ ]:
sub_df = pd.DataFrame()
sub_df['Id'] = test_df['Id']
sub_df['Pawpularity'] = predictions_nn

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)

**If you find this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels. 😊**